In [ ]:
conda install -c conda-forge imbalanced-learn

In [ ]:
conda install -c conda-forge boruta_py

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC

In [ ]:
#set display max column to see all column
pd.set_option("display.max_columns", None)
pd.get_option("display.max_columns")

#set display max column to see all column
pd.set_option("display.max_rows", 150)
pd.get_option("display.max_rows")

In [ ]:
app_train = pd.read_csv(r'C:\Users\Admin\Documents\GitHub\CreditRiskML2\data/loan1.csv')

In [ ]:
featurenames =    ['Creditability', 'Account_Balance', 'Duration_of_Credit_monthly', 'Payment_Status_of_Previous_Credit', 'Purpose	Credit_Amount', 'Value_Savings_Stocks', 'Length_of_current_employment', 'Instalment_per_cent', 'Sex_Marital_Status', 'Guarantors', 'Duration_in_Current_address', 'Most_valuable_available_asset', 'Age_years', 'Concurrent_Credits', 'Type_of_apartment', 'No_of_Credits_at_this_Bank', 'Occupation', 'No_of_dependents', 'Telephone', 'Foreign_Worker']

In [ ]:
app_train.tail()

In [ ]:
app_train["Payment_Status_of_Previous_Credit"].value_counts()


In [ ]:
app_train['Payment_Status_of_Previous_Credit'].astype(int).plot.hist();

In [ ]:
print(app_train.dtypes)

In [ ]:
#looking for broken data
app_train.describe()

In [ ]:
featurenames =    ['Creditability','Account_Balance', 'Duration_of_Credit_monthly', 'Payment_Status_of_Previous_Credit',  'Credit_Amount', 'Value_Savings_Stocks',  'Length_of_current_employment',
   'Most_valuable_available_asset', 'Age_years', 'No_of_Credits_at_this_Bank', 'Occupation', 'Foreign_Worker']

In [ ]:
def yfromX(X):
    y = X['Creditability'] + X['Duration_of_Credit_monthly']**2 + np.sin(3 * X['Account_Balance']) + (X['Payment_Status_of_Previous_Credit'] * X['Credit_Amount'] * X['Age_years'])
    return y

In [ ]:
# creating xy
np.random.seed(0)

X = pd.DataFrame(np.random.normal(size = (20000, len(featurenames))), columns = featurenames)
y = yfromX(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:

##Data that is important
importantstuff = list(X.columns[boruta.support_])
print(f"stuff that is important: {importantstuff}")
##Data that is important but not confirmed
maybeimportant = list(X.columns[boruta.support_weak_])
print(f"stuff that might be important: {maybeimportant}")
##Just garbage...
uselessstuff = list(X.columns[~(boruta.support_ | boruta.support_weak_)])
print(f"stuff that is just useless: {uselessstuff}")

In [ ]:
np.random.seed(0)

X = pd.DataFrame(np.random.normal(size = (20000, len(featurenames))), columns = featurenames)
y = yfromX(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:
#need to chose estimator obv
estimator = RandomForestRegressor()

In [ ]:
# Find correlation with the target and sort
correlations = app_train.corr()['Creditability'].sort_values()

# Display correlations
print('Postive Correlations:\n', correlations.tail(15))
print('\nNegative Correlations:\n', correlations.head(15))

In [ ]:
#object n stuff
boruta = BorutaPy(estimator = estimator, n_estimators = 'auto', max_iter = 100)
boruta.fit(np.array(X_train), np.array(y_train))

In [ ]:
app_train_selected = app_train[['Duration_in_Current_address', 'No_of_dependents', 'Type_of_apartment', 'Guarantors', 'Telephone', 'No_of_Credits_at_this_Bank', 'Foreign_Worker', 'Sex_Marital_Status', 'Age_years', 'Concurrent_Credits', 'Length_of_current_employment', 'Value_Savings_Stocks', 'Payment_Status_of_Previous_Credit', 'Account_Balance']]

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size = .3, random_state = 10)

In [ ]:

new_rf = RandomForestRegressor(n_jobs = -1, max_depth = 5)

boruta_selector = BorutaPy(new_rf, n_estimators = 'auto', random_state = 0)
boruta_selector.fit(np.array(X_train), np.array(y_train))

boruta_ranking = boruta_selector.ranking_
selected_features = np.array(featurenames)[boruta_ranking <= 2]

In [ ]:
boruta_ranking = pd.DataFrame(data=boruta_ranking, index=X_train.columns.values, columns=['values'])
boruta_ranking['Variable'] = boruta_ranking.index
boruta_ranking.sort_values(['values'], ascending=True, inplace=True)

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
ax = sns.barplot(x='values',y='Variable',data=boruta_ranking, color='b')
plt.title('Boruta Feature Ranking')
plt.xlabel('')
plt.ylabel('')
plt.tight_layout()

In [ ]:
# The output from Boruta is a ranking number (Rank 1: confirmed, Rank 2: moderate influence, Above Rank 3: Rejected). 
# As a result, we can group them according to need. Every attribute with a rank of 1 denotes a verified impact on the target variable (creditability). 
# A rating of 2 implies some influence. Anything over 3 can be disregarded because it has no influence.
# We selected all the appropriate dependent variables using Boruta, and we eliminated the noise.
# Boruta can be a very effective tool for comparing the relevance of features using traditional approaches.